Reference: 
From Random Polygon to Ellipse: An Eigenanalysis, Adam N. Elmachtoub, Charles F. van Loan.

This python notebook is devoted to the paper and to reproducing the experiments.

**Python 3**, **NumPy** and **Matplotlib** are required for the environment.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
valuetype = 'float64'
n = 15
xs = np.zeros((n,1),dtype=valuetype)
ys = np.zeros((n,1),dtype=valuetype)
M  = np.zeros((n,n),dtype=valuetype)

# initialize the matrices according to the paper
def initialize(size = 0,seed = 0):
    global xs, ys, M, n
    if size > 0: n = size
    np.random.seed(seed)
    xs = np.random.random((n,1)).astype(valuetype)-0.5
    ys = np.random.random((n,1)).astype(valuetype)-0.5
    
    xs -= np.mean(xs)
    ys -= np.mean(ys)

    # adjency matrix
    M = np.zeros((n,n),dtype=valuetype)
    for i in range(n):
        M[i][i] = 0.5
        M[i][(i+1)%n] = 0.5
        
# plot the points conveniently
def plotter(xs,ys,color='b',line=True,point=True,title=None,new=True,show=True):
    if new:
        plt.figure()
    plt.gca().set_aspect(1)
    if type(xs) != list:
        if point:
            plt.scatter(xs,ys,color=color)
        if line:
            xs = xs.flatten().tolist()
            ys = ys.flatten().tolist()
            xs.append(xs[0])
            ys.append(ys[0])
            plt.plot(xs,ys,color=color)
    else:
        for i in range(len(xs)):
            if point:
                plt.scatter(xs[i],ys[i],color=color[i])
            if line:
                xs[i] = xs[i].flatten().tolist()
                ys[i] = ys[i].flatten().tolist()
                xs[i].append(xs[i][0])
                ys[i].append(ys[i][0])
                plt.plot(xs[i],ys[i],color=color[i])
    plt.xlim(-0.6,0.6)
    plt.ylim(-0.6,0.6)
    if title:
        plt.title(title)
    if show:
        plt.show()

1.2 Iteration: A First Try

Generate a random set of points as a polygon. Replace them by the midpoints of the edges in each iteration. Repeat this and see the polygon gradually untangle and converge to a single point.

In [ ]:
initialize(15)
keystage = [0,5,20,100]
for epoch in range(101):
    if epoch in keystage:
        plotter(xs,ys,title='After %d Averagings'%epoch)
        plt.show()
    xs = M @ xs
    ys = M @ ys 

Let $k_n$ be the smallest $k$ such that after $k$ averagings the edges untangle. Here are some average $k_n$ s corresponding to various $n$ s.

<font color='red'>WARNING</font> This experiment is so slow that we introduce the magical numba-jit accelerator. This sharply cuts the executing time from several minutes down to 4 seconds. If you have not installed **numba**, it is recommended to use the command **pip install numba** to install one. 

It is an alternative to comment off the jit-decorator before these functions if numba is unavailable, but the following code will possibly runs a long time.

In [ ]:
from numba import jit
    
@jit(nopython = True)
def between(x1,x2,x3):
    return min(x1,x3) < x2 < max(x1,x3)

# check if the segements {(x1,y1),(x2,y2)} and {(x3,y3),(x4,y4)} intersect
@jit(nopython = True)
def intersects(x1,y1,x2,y2,x3,y3,x4,y4):
    if x1 != x2:
        k1 = (y2-y1)/(x2-x1)
        b1 = y1 - k1*x1 
        if x3 != x4:
            k2 = (y4-y3)/(x4-x3)
            b2 = y3 - k2*x3
            if k1 == k2 :
                return False
            # k1x + b1 = k2x + b2
            x = (b2 - b1)/(k1 - k2)
            return between(x1,x,x2)  and between(x3,x,x4)
        else:
            y = k1*x3 + b1
            return between(x1,x3,x2) and between(y3,y,y4)
    else:
        if x3 != x4:
            k2 = (y4-y3)/(x4-x3)
            b2 = y3 - k2*x3
            y = k2*x1 + b2
            return between(x3,x1,x4) and between(y1,y,y2)
    return False

# check whether the figure untangles by enumerating all the combinations of the edges
@jit(nopython = True)
def check_untangle(n,xs,ys):
    # to form a cycle
    # extract each pair of edges
    for i in range(n):
        for j in range(n):
            if i == j or i == j - 1 or i == j + 1: continue
            if intersects(xs[i][0],ys[i][0],xs[(i+1)%n][0],ys[(i+1)%n][0],
                            xs[j][0],ys[j][0],xs[(j+1)%n][0],ys[(j+1)%n][0]):
                return False
    return True

# return the minimal k such that the figure untangles
@jit(nopython = True)
def untangle(n,M,xs,ys):
    for epoch in range(0,2001):
        if check_untangle(n,xs,ys):
            return epoch
        xs = M @ xs
        ys = M @ ys
    return epoch

In [ ]:
ns = [10,20,40,80]
ks = []
examples = 100
startseed = 0
for n in ns:
    k = 0
    for seed in range(startseed, startseed + examples):
        initialize(n,seed)
        k += untangle(n,M,xs,ys)
    ks.append(k * 1.0 / examples)
print('Average k = ' , ks)


1.3. Iteration: A Second Try

In this experiment, vector normalization is included so that the figure does not collapse to the centroid. The result converges to an ellipse with 45-degree tilt from the coordinate axes.

In [ ]:
initialize(15)
for epoch in range(100):
    xs = M @ xs
    ys = M @ ys 
    xs /= np.linalg.norm(xs)
    ys /= np.linalg.norm(ys)
plotter(xs,ys,show=False)

coor = np.linspace(-0.6,0.6,num=1000)
plt.plot(coor,coor,'gray')
plt.plot(coor,-coor,'gray')
plt.show()

3.1. An Experiment and Another Conjecture.

Even-indexed polygons given by the following code are all the same, while the odd-indexed are all the same. The even polygon is marked red while the odd marked blue.

In [ ]:
initialize(12)
np.random.seed(1)
theta_u = np.random.random() * 2*np.pi
theta_v = np.random.random() * 2*np.pi
cosx = math.sqrt(2.0/n) * np.cos(np.linspace(0,2.0*(n-1)*np.pi/n,num=n))
sinx = math.sqrt(2.0/n) * np.sin(np.linspace(0,2.0*(n-1)*np.pi/n,num=n))

u = math.cos(theta_u)*cosx + math.sin(theta_u)*sinx
v = math.cos(theta_v)*cosx + math.sin(theta_v)*sinx
us , vs = [] , []
us.append(u)
vs.append(v)
for epoch in range(11):
    u = M @ u 
    v = M @ v
    u /= np.linalg.norm(u)
    v /= np.linalg.norm(v)
    us.append(u)
    vs.append(v)
plotter([us[-1],us[-2]],[vs[-1],vs[-2]],['r','b'])

4.2. The D2 Ellipse.

Calculation gives that the points form a 45-degree ellipse with semiaxes $|\sigma_1|, |\sigma_2|$. 
The equation is given by $\frac{(x-y)^2}{2\sigma_1^2} + \frac{(x+y)^2}{2\sigma_2^2}=1$. 

The code below draws the ellipse (in green) first and then the points to verify the latter lie precisely on the ellipse.

In [ ]:
initialize(12)
sigma1 = 2/math.sqrt(n) * math.cos((theta_v - theta_u)/2)
sigma2 = 2/math.sqrt(n) * math.sin((theta_v - theta_u)/2)

# non-tilt ellipse
base = np.linspace(0, 2*np.pi, num=1000)
ellipse_x = sigma1 * np.cos(base)
ellipse_y = sigma2 * np.sin(base)

# spin 45-degree
ellipse_x , ellipse_y = (ellipse_x + ellipse_y)/math.sqrt(2),  (ellipse_x - ellipse_y)/math.sqrt(2) 
plotter(ellipse_x,ellipse_y,color='g',point=False,show=False)
plotter([us[-1],us[-2]],[vs[-1],vs[-2]],['r','b'],line=False,new=False)
plt.show()

4.4. Alternative Normalizations.

Use different vector norms and chech how the points converge. The following experiment checks that 
the figure $P_{2n}$ coincides with $P_0$, indicating the iteration has a cycle of $2n$ (regardless of the order of norms because normalization does not influence their directions).

In [ ]:
initialize()
cosx = math.sqrt(2.0/n) * np.cos(np.linspace(0,2*(n-1)*np.pi/n,num=n))
sinx = math.sqrt(2.0/n) * np.sin(np.linspace(0,2*(n-1)*np.pi/n,num=n))
np.random.seed(1)
u = (np.random.random()-0.5)*cosx + (np.random.random()-0.5)*sinx
v = (np.random.random()-0.5)*cosx + (np.random.random()-0.5)*sinx

# order of norm
ordx = 3
ordy = 1.5

u /= np.linalg.norm(u, ord = ordx)
v /= np.linalg.norm(v, ord = ordy)
u0 , v0 = u.copy() , v.copy()
for epoch in range(2*n):
    u = M @ u 
    v = M @ v
    u /= np.linalg.norm(u, ord = ordx)
    v /= np.linalg.norm(v, ord = ordy)
plotter([u0,u],[v0,v],color=['r','b'])